In [1]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm

### Load data

In [2]:
# By uncommenting the lines below we allow for "eager execution"
# which is useful for debugging, as every command is executed immediately
#
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()

### Model setup

In [3]:
first_time_seen = [0, 1, 0, 4]
f = tf.constant(first_time_seen, name="first_time_seen")
f

<tf.Tensor: id=0, shape=(4,), dtype=int32, numpy=array([0, 1, 0, 4], dtype=int32)>

In [4]:
last_time_seen = [0, 2, 4, 4]
l = tf.constant(last_time_seen, name="last_time_seen")
l

<tf.Tensor: id=2, shape=(4,), dtype=int32, numpy=array([0, 2, 4, 4], dtype=int32)>

In [5]:
count = [1, 2, 5, 1]
u = tf.constant(count, name="count", dtype=tf.float64)
u

<tf.Tensor: id=4, shape=(4,), dtype=float64, numpy=array([1., 2., 5., 1.])>

In [6]:
I = len(count)
K = max(last_time_seen)+1

In [7]:
# The number of unseen workers.
Uh  = tf.get_variable( dtype=tf.float64, shape=(), name = "u_hidden", initializer=tf.constant_initializer(10) ) 
U = tf.square(Uh, name="unseen")

In [8]:
# Propensity distribution parametes (propensity ~ Beta(a,b))
a = tf.get_variable("pronensity_alpha", shape=(), dtype=tf.float64, initializer=tf.constant_initializer(np.sqrt(2))  )
a = tf.square(a)
b = tf.get_variable("pronensity_beta",  shape=(), dtype=tf.float64, initializer=tf.constant_initializer(np.sqrt(2)) )
b = tf.square(b)

In [9]:
# Arrival and departure rates
# In our likelihood lambda_s and mu_q are treated as probabilities and cannot be negative
# To ensure non-negativity, and put the values between 0..1, we will start with two unconstrained
# parameters, which we will then convert into a 0..1 variable using a tanh. 
# Then, we can use the standard unconstrained optimization on lh and mh
# We initialize variables to log(1/(K-1)) to out all variables to be 
lh = tf.get_variable("arrivals_latent",   [K], dtype=tf.float64, initializer=tf.constant_initializer(np.log(1/(K-1))))
mh = tf.get_variable("departures_latent", [K], dtype=tf.float64, initializer=tf.constant_initializer(np.log(1/(K-1))))

# Ensures the values lambda and mu are positive and between 0 and 1
lmd = tf.sigmoid(lh, name='arrivals')
mu  = tf.sigmoid(mh, name='departures')

In [10]:
lmd

<tf.Tensor: id=51, shape=(5,), dtype=float64, numpy=array([0.2, 0.2, 0.2, 0.2, 0.2])>

In [11]:
mu

<tf.Tensor: id=53, shape=(5,), dtype=float64, numpy=array([0.2, 0.2, 0.2, 0.2, 0.2])>

### Vectorizing the implementation of likelihood

### $L_i = \sum_{s=1}^{f_i} \sum_{q=l_i}^K \lambda_s \cdot  \left( \prod_{v=s}^{q-1} \left(1 - \mu_v\right) \right) \cdot \mu_q \cdot  R_i(s,q) $

###  $R_i(s,q) = f(u_i|n,\alpha,\beta) = \frac{\Gamma(n+1)}{\Gamma(u_i+1)\Gamma(n-u_i+1)} \frac{\Gamma(u_i+\alpha)\Gamma(n-u_i+\beta)}{\Gamma(n+\alpha+\beta)}                        \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)}$

with $n = q-s$ (or $n = q-s+1$?)


#### Computing the Birth-Death probability for each worker

We will start by computing the probability that a given worker arrived at time $s$ and departed at time $q$.

We will store the result at a $I \times K \times K$ tensor `LD_isq`. The values in this tensor will be the product $\lambda_s \cdot  \left( \prod_{v=s}^{q-1} \left(1 - \mu_v\right) \right) \cdot \mu_q$, and will be zero for infeasible combinations of $s$ and $q$. For example, if a user was first seen at time $f$, any arrival time $s>f$ is infeasible. Similarly, for departure times.

In [12]:
# To allow for the sums of variable length s .. f_i and l_i...K, we will use binary masks
# This allows the products to be defined over a constant set of dimensions (ie. 1..K instead of 1..fi)

In [13]:
# Create masks using the values from f so that for each worker we add the lamda_s until the first appearance
s_mask = tf.sequence_mask(f+1, K)
s_mask = tf.cast(s_mask, tf.float64)

# Create masks using the values from l so that for each worker we add the mu_q from the first appearance until the end
q_mask = ~tf.sequence_mask(l, K)
q_mask = tf.cast(q_mask, tf.float64)

# This is a mask that only allows feasible combinations of s and q for a given worker i
# It is effectively the outer product of the s_mask and q_mask, for each worker
i_mask = tf.einsum("is,iq->isq", s_mask,q_mask)
i_mask

<tf.Tensor: id=80, shape=(4, 5, 5), dtype=float64, numpy=
array([[[1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 1., 1., 1.],
        [0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.]]])>

In [14]:
# And create a mask to calculate the product of 1-mu_v
# It has as 1 the mu's that need to be in the product, 
# for arrival at s and departure at q
v = np.zeros([K,K,K])
for s in np.arange(K):
    for q in np.arange(s,K):
        c1 = np.zeros(s)
        c2 = np.ones(q-s)
        c3 = np.zeros(K-q)
        v[s,q]= np.concatenate((c1, c2, c3))
v_mask = tf.constant(v)
v_mask

<tf.Tensor: id=82, shape=(5, 5, 5), dtype=float64, numpy=
array([[[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 1., 0., 0.],
        [0., 1., 1., 1., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 1., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])>

In [15]:
# Calculating m_v(s,q) = Prod_s^q-1 mu_v
# mu_v[s,q]: Probability of surviving from period s to period q-1
# if q-1<s then probability is 1
mu_v = tf.pow(1-mu, v_mask)
mu_v = tf.reduce_prod(mu_v, axis=2)
mu_v

<tf.Tensor: id=88, shape=(5, 5), dtype=float64, numpy=
array([[1.    , 0.8   , 0.64  , 0.512 , 0.4096],
       [1.    , 1.    , 0.8   , 0.64  , 0.512 ],
       [1.    , 1.    , 1.    , 0.8   , 0.64  ],
       [1.    , 1.    , 1.    , 1.    , 0.8   ],
       [1.    , 1.    , 1.    , 1.    , 1.    ]])>

In [16]:
# This is a SxQ (ie, KxK) matrix that describes the likelihood of arriving 
# at time s and departing at time q
LD = tf.einsum('s,sq,q->sq', lmd, mu_v, mu)
LD

<tf.Tensor: id=103, shape=(5, 5), dtype=float64, numpy=
array([[0.04    , 0.032   , 0.0256  , 0.02048 , 0.016384],
       [0.04    , 0.04    , 0.032   , 0.0256  , 0.02048 ],
       [0.04    , 0.04    , 0.04    , 0.032   , 0.0256  ],
       [0.04    , 0.04    , 0.04    , 0.04    , 0.032   ],
       [0.04    , 0.04    , 0.04    , 0.04    , 0.04    ]])>

In [17]:
LD_isq = tf.einsum('sq,isq->isq', LD, i_mask)
LD_isq

<tf.Tensor: id=112, shape=(4, 5, 5), dtype=float64, numpy=
array([[[0.04    , 0.032   , 0.0256  , 0.02048 , 0.016384],
        [0.      , 0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      , 0.      ]],

       [[0.      , 0.      , 0.0256  , 0.02048 , 0.016384],
        [0.      , 0.      , 0.032   , 0.0256  , 0.02048 ],
        [0.      , 0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      , 0.      ]],

       [[0.      , 0.      , 0.      , 0.      , 0.016384],
        [0.      , 0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      , 0.      ],
        [0.      , 0.      , 0.      , 0.      , 0.      ]],

       [[0.      , 0.      , 0.    

#### Vectorizing the Beta-Binomial computation

Once we know the arrival-departure likelihoods for each user, we move on to vectorize the calculation of the Beta-Binomial likelihood $R_i(s,q)$.

We know that 

###  $R_i(s,q) = f(u_i|n,\alpha,\beta) = \frac{\Gamma(n+1)}{\Gamma(u_i+1)\Gamma(n-u_i+1)} \frac{\Gamma(u_i+\alpha)\Gamma(n-u_i+\beta)}{\Gamma(n+\alpha+\beta)}                        \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)}$

with $n = q-s+1$.

In [28]:
# n = q - s + 1
d = np.zeros([K,K])
for s in range(K):
    for q in range(K):
        d[s,q] = q-s+1
        
# n-u
# count = list(data['count'].values)
n_u = np.zeros([I,K,K])
for i in range(len(count)):
    n_u[i] = d - count[i]

n = tf.constant(np.clip(d,a_min=0,a_max=K+1))
n_u = tf.constant(np.clip(n_u,a_min=0,a_max=max(count)))

In [73]:
def transform_1d_to_3d(x, K, L):
    '''
    Takes vector x of dimensionality I and broadcast it, to return a 3d tensor,
    I x K x L, where the values of the 2d KxL matrix have the values of x[i]
    '''
    return tf.einsum("i,sq->isq", x, tf.ones(shape=(K,L), dtype=tf.float64) )

transform_1d_to_3d_new(u, K, K)

<tf.Tensor: id=670, shape=(4, 5, 5), dtype=float64, numpy=
array([[[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]],

       [[5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]])>

In [31]:
n

<tf.Tensor: id=148, shape=(5, 5), dtype=float64, numpy=
array([[1., 2., 3., 4., 5.],
       [0., 1., 2., 3., 4.],
       [0., 0., 1., 2., 3.],
       [0., 0., 0., 1., 2.],
       [0., 0., 0., 0., 1.]])>

In [72]:
def transform_2d_to_3d(xy, Z):
    '''
    Takes matrix  xy of dimensionality X * Y  and broadcast it Z times, 
    to return a 3d tensor, Z x X x Y, where the values of the 2d KxL matrix have the values of xy[z]
    '''
    return tf.einsum("xy,z->zxy", xy, tf.ones(shape=[Z], dtype=tf.float64) )

transform_2d_to_3d(n , I)

<tf.Tensor: id=657, shape=(4, 5, 5), dtype=float64, numpy=
array([[[1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4.],
        [0., 0., 1., 2., 3.],
        [0., 0., 0., 1., 2.],
        [0., 0., 0., 0., 1.]],

       [[1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4.],
        [0., 0., 1., 2., 3.],
        [0., 0., 0., 1., 2.],
        [0., 0., 0., 0., 1.]],

       [[1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4.],
        [0., 0., 1., 2., 3.],
        [0., 0., 0., 1., 2.],
        [0., 0., 0., 0., 1.]],

       [[1., 2., 3., 4., 5.],
        [0., 1., 2., 3., 4.],
        [0., 0., 1., 2., 3.],
        [0., 0., 0., 1., 2.],
        [0., 0., 0., 0., 1.]]])>

## $\frac{\Gamma(n+1)}{\Gamma(u_i+1)\Gamma(n-u_i+1)} \frac{\Gamma(u_i+\alpha)\Gamma(n-u_i+\beta)}{\Gamma(n+\alpha+\beta)}                        \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)}$

In [ ]:
# We compute the log of the value above, as Tensorflow does not 
# have Gamma function implemented directly but it has lgamma 
# Matrix of shape I x K (s=0..K-1)  x K (q=0..K-1)  

# tf.lgamma(n+1) 
R_isq = transform_2d_to_3d( tf.lgamma(tf.constant(n+1) ), I)
# - tf.lgamma(u+1) 
R_isq -= transform_1d_to_3d( tf.lgamma(u+1), K, K )
# - tf.lgamma(n-u+1)
R_isq -= tf.lgamma( tf.constant(n_u) + 1 )   
# + tf.lgamma(u+a) 
R_isq +=  transform_1d_to_3d(tf.lgamma(u+a), K, K)
# + tf.lgamma(n-u+b)
R_isq +=  tf.lgamma( tf.constant(n_u) + b )
# - tf.lgamma(n+a+b)
R_isq -= transform_2d_to_3d( tf.lgamma(tf.constant(n) +a+b ), I)
# + tf.lgamma(a+b - tf.lgamma(a) -tf.lgamma(b)
R_isq += tf.lgamma( a + b ) - tf.lgamma(a) - tf.lgamma(b )

# The above is the computation of the log, so we take the exponent
R_isq = tf.exp(R_isq)

In [ ]:
# This is the likelihood, that the worker i, who arrived at time s and departed at time q
# also appeared u_i times in our surveys.
R_isq

In [ ]:
u, f ,l

In [ ]:
R_isq[1] # this is the likelihood matrix for the user that appeared twice. 
# Elements in the diagonal and below (inclusive) are not valid, as they allow for s-q+1 to be 1 and below


In [ ]:
R_isq[2] # this is the likelihood matrix for the user that appeared five times. 
# only the element R_isq[2][0][4] is a valid element, as it is the only one where s-q+1 >= 5


#### Computing the likelihood



In [ ]:
LD_isq * R_isq

In [ ]:
# We use einsum to calculate the product of LD_isq with R_isq
# and then take the sums of likelihoods for different workers,
# sum across s and q, and return a vector with I elements
Li = tf.einsum('isq,isq->i', LD_isq, R_isq)

# Li =  tf.reduce_sum(products)
Li

-----------------

### Likelihood for the never observed workers:

## $L_0 = \sum_{s=1}^{K} \sum_{q=s}^K \lambda_s \cdot \left( \prod_{v=s}^{q-1} \left( 1 - \mu_v \right) \right) \cdot \mu_q \cdot  \frac{\Gamma(n+\beta)}{\Gamma(n+\alpha+\beta)}             \frac{\Gamma(\alpha+\beta)}{\Gamma(\beta)} $

### $n=q-s+1$


In [ ]:
LD0 = tf.einsum('s,sq,q->sq', lmd, mu_v, mu)
LD0

In [36]:
v0 = np.zeros([K,K])
for s in np.arange(K):
    c1 = np.zeros(s)
    c2 = np.ones(K-s) 
    v0[s]= np.concatenate((c1, c2))
v0_mask = tf.constant(v0)
v0_mask

<tf.Tensor: id=239, shape=(5, 5), dtype=float64, numpy=
array([[1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1.],
       [0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.]])>

In [ ]:
# Eliminate from the matrix the combinations of s/q that do not appear
# in the sum above
LD0 = tf.multiply( LD0  , v0_mask )
LD0

In [ ]:
R0_sq = tf.exp(
 tf.lgamma(tf.constant(n) + b )-tf.lgamma(tf.constant(n) +a + b )
+ tf.lgamma(a + b )-tf.lgamma( b )
)
R0_sq

In [ ]:
LD0*R0_sq

In [ ]:
# Do an element-wise product of the three matrices (as they all have the same dimensions)
# and then sum all the elements
L0 = tf.einsum('sq,sq->', LD0, R0_sq)
# L0 = tf.reduce_sum(products0)
L0

--------------------

### Setting up the optimization objective

In [ ]:
# Once we have the individual likelihoods, we take the sum of their logs
# In our case N = I + U (all workers) and N - D = U (unseen workers)
# We use that Γ(n+1) = n!  and logGamma(n+1) = log(n!)

# We want sum(lambda)=1 and sum(mu)=1
obj1 = tf.square(tf.reduce_sum(lmd) - 1) + tf.square(tf.reduce_sum(mu) - 1)

# These are the likelihood function
obj2 = tf.reduce_sum(tf.log(Li))
obj3 = U * tf.log(L0)
obj4 = tf.lgamma(I+U+1.0) - tf.lgamma(U+1.0) - tf.lgamma( tf.constant(I, dtype=tf.float64) +1.0)

objective = 100000*obj1 - (obj4 + obj2 + obj3) 
objective

In [ ]:
train_step = tf.train.AdamOptimizer(learning_rate=0.01).minimize(objective)

### Training the model

In [ ]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
loss = []

feed_data = {
    f: np.array(data['min']),  
    l: np.array(data['max']), 
    u: np.array(data['count'])
}

In [ ]:
_, obj_value, a_est, b_est, lmd_est, mu_est, U_est, o1, o2, o3 = sess.run([train_step, 
                                                               objective, 
                                                               a, 
                                                               b, 
                                                               lmd, 
                                                               mu, 
                                                               U,
                                                              obj1, obj2, obj3], feed_dict=feed_data)

In [ ]:
o1, o2, o3

In [ ]:
Rsqu.eval( feed_dict=feed_data)

In [ ]:
t = tqdm(range(10000))

feed_data = {
    f: np.array(data['min']),  
    l: np.array(data['max']), 
    u: np.array(data['count'])
}

for epoch in t: 
    _, obj_value, a_est, b_est, lmd_est, mu_est, U_est = sess.run([train_step, objective, a, b, lmd, mu, U], feed_dict=feed_data)
    
    
    avg_lambda = np.mean(lmd_est)
    avg_mu = np.mean(mu_est) 
    std_lambda = np.std(lmd_est)
    std_mu = np.std(mu_est)
    loss.append( {"Iteration": epoch, "Loss": obj_value, "a_est": a_est, "b_est": b_est, "U": U_est } )
    template = "{obj:10.3f} / {alpha:6.3f} / {beta:6.3f} / {al:6.3f} / {am:6.3f} / {u:6.3f}"
    if epoch % 100 == 0:
        status = template.format(
                        obj = obj_value, 
                        alpha = a_est, 
                        beta = b_est, 
                        al = avg_lambda, 
                        am = avg_mu, 
                        u = U_est)
        t.set_description(status)


In [ ]:
pd.DataFrame(loss).set_index('Iteration').Loss.plot()

In [ ]:
pd.DataFrame(loss).set_index('Iteration')[ ['a_est','b_est'] ].plot()

In [ ]:
pd.DataFrame(loss).set_index('Iteration')[ ['U'] ].plot()